Import the necessary libraries and dependencies

In [1]:
from python_lib.modules import *
import torch
import numpy as np
import os
from python_lib.saveasfile import *

c:\Users\chiny\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\chiny\anaconda3\envs\pytorch\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://pytorch.org/audio/stable/index.html
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://pytorch.org/audio/stable/index.html


# Model initialisation 

Use BlockSave to save all the weights into a `.bin` file

Be sure to `load_state_dict` from a `.pt`/`.pth` file before BlockSave

It's omitted here as I don't have the weights on hand, so I just use a random initialised weights.

In [2]:
model = ECAPA_TDNN(input_size = 2, channels=[8,8,8,8,16], lin_neurons=6, device = "cuda").to("cuda")

BlockSave(model.return_layers(), "fullecapa", "ECAPAweights")

# Generate Input Feats for testing purposes

In [6]:
input_feats = torch.rand([10,2,64]).to("cuda")
input_feats_np = input_feats.cpu().detach().numpy()
# input_feats_np = np.reshape(input_feats_np, (10,2,64))
print(input_feats_np)
dim = input_feats_np.shape
print(dim)

flatten_inputs = input_feats_np.flatten()
with open(os.path.join("ECAPAweights", f"ecapainput_2x64_10.bin"), "wb") as f:
            # Write the dimensions down
            f.write(np.array(dim, dtype=np.int32).tobytes())
            # Write the flatten bias down
            f.write(flatten_inputs.tobytes())

[[[0.023166   0.9731417  0.7460158  ... 0.46480352 0.21934539 0.42179698]
  [0.10872841 0.282017   0.6692086  ... 0.04658151 0.9434326  0.47276652]]

 [[0.324135   0.65740377 0.35862434 ... 0.4081254  0.22546905 0.5897336 ]
  [0.10112667 0.23391163 0.8612445  ... 0.25318027 0.73377275 0.42561293]]

 [[0.79551876 0.6762347  0.17714351 ... 0.27255458 0.6157144  0.8788035 ]
  [0.8718786  0.4286331  0.8683653  ... 0.72967803 0.8689484  0.40264505]]

 ...

 [[0.45201188 0.8362149  0.42966342 ... 0.5648378  0.9012359  0.88712066]
  [0.19112778 0.34422404 0.7041239  ... 0.04327422 0.83097804 0.66088027]]

 [[0.8094985  0.34216088 0.45539242 ... 0.4457382  0.16241944 0.1417619 ]
  [0.55029905 0.3897081  0.8553558  ... 0.44461846 0.00520104 0.90155476]]

 [[0.11795723 0.6111558  0.43313897 ... 0.26249826 0.41432053 0.60882384]
  [0.6649869  0.6734526  0.43903393 ... 0.13070625 0.33854437 0.22053027]]]
(10, 2, 64)


# Run the model using eval/train mode

In [4]:
model.eval()

model(input_feats)

tensor([[0.1725, 0.1615, 0.1514, 0.1935, 0.1721, 0.1489],
        [0.1726, 0.1610, 0.1504, 0.1945, 0.1721, 0.1494],
        [0.1723, 0.1612, 0.1504, 0.1948, 0.1719, 0.1494],
        [0.1722, 0.1611, 0.1503, 0.1949, 0.1720, 0.1494],
        [0.1724, 0.1607, 0.1505, 0.1947, 0.1720, 0.1496],
        [0.1721, 0.1615, 0.1507, 0.1945, 0.1720, 0.1494],
        [0.1725, 0.1613, 0.1504, 0.1946, 0.1717, 0.1495],
        [0.1726, 0.1611, 0.1506, 0.1943, 0.1720, 0.1493],
        [0.1727, 0.1607, 0.1510, 0.1944, 0.1718, 0.1495],
        [0.1724, 0.1611, 0.1513, 0.1943, 0.1716, 0.1493]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)